# Azure AI Agents with Model Context Protocol (MCP) Support

This notebook demonstrates how to use Azure AI Agents with Model Context Protocol (MCP) tools to create an intelligent agent that can leverage external MCP servers for enhanced capabilities.

## Install Required NuGet Packages

First, we need to install the Azure AI Agents Persistent package which provides the core functionality for working with Azure AI Agents.

## Keyless Authentication Benefits

This notebook demonstrates **keyless authentication** which provides several advantages:
- ✅ **No API keys to manage** - Uses Azure identity-based authentication
- ✅ **Enhanced security** - No secrets stored in code or configuration
- ✅ **Automatic credential rotation** - Azure handles credential lifecycle
- ✅ **Role-based access** - Uses Azure RBAC for fine-grained permissions

The `DefaultAzureCredential` will automatically use the best available credential source:
1. Managed Identity (when running in Azure)
2. Azure CLI credentials (during development)
3. Visual Studio credentials
4. Environment variables (if configured)

In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Install the Azure Identity package for authentication with Azure services using DefaultAzureCredential.

In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Import Required Namespaces

Import the necessary namespaces for Azure AI Agents and Azure Identity.

In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Configure Azure AI Agent Client (Keyless Authentication)

Set up the configuration variables and create the PersistentAgentsClient using **keyless authentication**:
- **projectEndpoint**: The Azure AI Foundry project endpoint
- **modelDeploymentName**: The name of the deployed AI model (GPT-4.1 nano)
- **mcpServerUrl**: The URL of the MCP server (Microsoft Learn API)
- **mcpServerLabel**: A label to identify the MCP server
- **DefaultAzureCredential**: Uses managed identity, Azure CLI, or other credential sources (no API keys required)

In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Create MCP Tool Definition

Create an MCP tool definition that connects to the Microsoft Learn MCP server. This will allow the agent to access Microsoft Learn content and documentation.

In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Create the AI Agent

Create a persistent AI agent with the specified model and MCP tools. The agent is configured with:
- The GPT-4.1 nano model
- Instructions to use MCP tools for assistance
- Access to the Microsoft Learn MCP server

In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Create Thread and Send Message

Create a conversation thread and send a user message asking about the difference between Azure OpenAI and OpenAI. This will test the agent's ability to use the MCP tools to provide accurate information.

In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Configure MCP Tool Resources (Keyless)

Set up the MCP tool resources. For a truly keyless approach, you can remove custom headers if the MCP server supports Azure identity-based authentication. The example below shows how to add headers if needed, but for keyless scenarios, these may not be required.

In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Start Agent Run

Create and start a run to process the user's message. The agent will use the configured MCP tools and resources to generate a response.

In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Monitor Run and Handle Tool Approvals (Keyless)

Monitor the agent run status and handle any required tool approvals. This loop:
1. Waits for the run to complete or require action
2. Automatically approves MCP tool calls when required
3. For keyless authentication, headers may not be needed if the MCP server supports Azure identity

In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Display Conversation Results

Retrieve and display all messages in the thread, showing both the user's question and the agent's response. The messages are displayed in chronological order with timestamps and role indicators.

In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi